In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
###Change the data file directory below appropriately
data = pd.read_csv('../raw_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv')
data.head()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,1449601597,1.000371,0.007671,-0.016173,0.027860,0.998221,1.000739,1.003265,0.891038,6.684582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,1449601657,1.000243,0.003782,-0.002713,0.007046,0.998463,1.000373,1.002088,1.647929,6.684605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,1449601717,1.000811,0.002082,-0.001922,0.003575,0.999653,1.000928,1.002032,1.960286,6.684610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,1449601777,1.001245,0.004715,-0.002895,0.008881,0.999188,1.001425,1.003500,1.614524,6.684601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,1449601855,1.001354,0.065186,-0.096520,0.165298,1.000807,1.002259,1.003631,0.837790,6.682252,...,0.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,0.0,2


# Interpolating acceleration columns with average values

In [3]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

# Splitting the data and loading it into a PyTorch dataloader

In [4]:
X = data.iloc[:,1:27]
y = data[['label:SITTING']]

X = X[y['label:SITTING'] == 1]
y = y[y['label:SITTING'] == 1]

X = interpolation(X).values
y = interpolation(y).values

print(len(X), len(y))

2253 2253


In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
#X

# Helper Functions for Training

In [19]:
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = 26, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, int(hidden_dim/2)),
            generator_block(int(hidden_dim/2), int(hidden_dim/4)),
            generator_block(int(hidden_dim/4), 30),
            generator_block(30, feature_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)

def discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Discriminator(nn.Module):
    def __init__(self, feature_dim = 26, hidden_dim = 16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator_block(feature_dim, hidden_dim),
            discriminator_block(hidden_dim, int(hidden_dim/2)),
            discriminator_block(int(hidden_dim/2), int(hidden_dim/4)),
            nn.Linear(int(hidden_dim/4), 1),
            nn.Sigmoid()                    
        )
    def forward(self, feature_vector):
        return self.disc(feature_vector)

def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    fake_features = gen(latent_vectors)
    pred_fake = disc(fake_features.detach())
    
    ground_truth = torch.zeros_like(pred_fake)
    loss_fake = criterion(pred_fake, ground_truth)
    
    pred_real = disc(real_features)
    ground_truth = torch.ones_like(pred_real)
    loss_real = criterion(pred_real, ground_truth)
    
    disc_loss = (loss_fake + loss_real) / 2
    return disc_loss

def get_gen_loss(gen, disc, criterion, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    fake_features = gen(latent_vectors)
    pred = disc(fake_features)
    gen_loss = criterion(pred, torch.ones_like(pred))
    return gen_loss

def visualize_gen_batch(gen, b_size, epochs = -1):
    #print(str(b_size))
    latent_vectors = get_noise(b_size, z_dim)
    #print(latent_vectors.shape)
    fake_features = gen(latent_vectors)
    #print(fake_features.shape)
    
    w_img = fake_features
    wmin = torch.min(w_img)
    wmax = torch.max(w_img)
    w_img = w_img.cpu()
    w_img = w_img.detach().numpy()
    c = plt.imshow(w_img, cmap ='Reds', vmin = wmin , vmax = wmax,
                        interpolation ='nearest', origin ='upper')
    plt.colorbar(c)
    plt.title('Generated Batch at Epoch ' + str(epochs), fontweight ="bold")
    plt.show()
    
def visualize_real_batch(features):
    w_img = features
    wmin = torch.min(w_img)
    wmax = torch.max(w_img)
    w_img = w_img.cpu()
    w_img = w_img.detach().numpy()
    c = plt.imshow(w_img, cmap ='Reds', vmin = wmin , vmax = wmax,
                        interpolation ='nearest', origin ='upper')
    plt.colorbar(c)
    plt.title('Real Batch of Data', fontweight ="bold")
    plt.show()
    
def performance_stats(gen, disc, b_size, batch = None):
    tp = 0
    fp = 0
    tn = 0
    fn = 0

    with torch.no_grad():
        if batch is None:
            latent_vectors = get_noise(b_size, z_dim)
            fake_features = gen(latent_vectors)
            y_hat = torch.round(disc(fake_features))
            y_label = [0] * b_size
            y_label = torch.Tensor(y_label)
        else:
            latent_vectors = get_noise(int(b_size/2), z_dim)
            fake_features = gen(latent_vectors)
            y_hat = torch.round(disc(fake_features))
            y_label = [0] * int(b_size/2)
            
            real_y_hat = torch.round(disc(batch[:int(b_size/2)]))
            for i in range(0, int(b_size/2)):
                y_label.append(1)
            y_hat = torch.cat((y_hat, real_y_hat), dim = 0)
            
            #print(y_hat)
            #print(y_label)
         
        
        for k in range(len(y_hat)):
            #True positive
            if y_label[k] == 1 and y_hat[k] == 1:
                tp += 1
            #False Negative
            elif y_label[k] == 1 and y_hat[k] == 0:
                fn += 1
            #True Negative
            elif y_label[k] == 0 and y_hat[k] == 0:
                tn += 1
            elif y_label[k] == 0 and y_hat[k] == 1:
                fp += 1
            
        class_acc = (tp + tn)/(tp + tn + fp + fn)
        
        if tp + fp == 0:
            precision = 0
        else:
            precision = tp / (tp + fp)
            
        if tp + fn == 0:
            recall = 0
        else:
            recall = tp / (tp + fn)
            
        if fp + tn == 0:
            fpR = 0
        else: 
            fpR = fp / (fp + tn)

        #print(f'Classification Accuracy: {class_acc:.2f}')
        #print(f'Precision: {precision:.2f}') #What percentage of a model's positive predictions were actually positive
        #print(f'Recall: {recall:.2f}') #What percent of the true positives were identified
        #print(f'F-1 Score: {2*(precision * recall / (precision + recall + 0.001)):.2f}')
        return class_acc, precision, recall, fpR, 2*(precision * recall / (precision + recall + 0.001))

# Model Hyperparameters (Always Run Again Before Starting Training Loop) 

In [24]:
# Loss function for model
criterion = nn.BCELoss()

# Digit Precision for printouts
dig = 3

# Max epochs to run
n_epochs = 3000

# Number of dimensions of latent vector
z_dim = 100

# Batch Size
batch_size = 200

# Learning Rates for Generator (Gen) and Discriminator (Disc)
gen_lr =  0.0001
disc_lr = 0.0001

# Constant epochs approach to train Discriminator, Generator
constant_train_flag = False # Set to true to train based on constant # of epochs per machine 
                           # Set to false to train dynamically based on machine performance
disc_epochs = 5            #Number of consecutive epochs to train discriminator before epoch threshold
gen_epochs = 2             #Number of consecutive epochs to train generator before epoch threshold
epoch_threshold = 50       #Epoch number to change training epoch ratio
disc_epochs_change = 1     #New number of consecutive epochs to train discriminator
gen_epochs_change = 50     #New number of consecutive epochs to train generator
rel_epochs = 0             # Epochs passed since last switch (always set to 0)


# Dynamic number of epochs to train Discriminator, Generator
static_threshold = 5   #Epoch number to change from static ratio to dynamic
static_disc_epochs = 3  #Number of consecutive epochs to train discriminator before epoch threshold
static_gen_epochs = 2   #Number of consecutive epochs to train generator before epoch threshold
pull_threshold = 0.75   #Accuracy threshold for switching machine training when the generator is no longer competitive
push_threshold = 0.50   #Accuracy threshold for switching machine training when the discriminator is no longer competitive

# Which machine to train (0 for Generator, 1 for Discriminator) !!!(do not change unless for good reason)!!!
GENERATOR = 0
DISCRIMINATOR = 1
to_train = DISCRIMINATOR
train_string = "DISC"

# Show model performance per batch (will always show summary for each epoch)
print_batches = False

# Moving corpus data into a pyTorch format !!!(do not change unless for good reason)!!!
train_features = torch.tensor(X)
train_labels = torch.tensor(y)
train_data = torch.utils.data.TensorDataset(train_features, train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)

# Initializiing the Machines !!!(do not change unless for good reason)!!!
disc = Discriminator()
gen = Generator(z_dim)
opt_disc = optim.Adam(disc.parameters(), lr = disc_lr)
opt_gen = optim.Adam(gen.parameters(), lr = gen_lr)

# Training Loop

In [25]:
last_real_features = []

for epoch in range(n_epochs):  
    if constant_train_flag:
        if to_train == DISCRIMINATOR and rel_epochs >= disc_epochs:
            rel_epochs = 0
            to_train = GENERATOR
            train_string = "GEN"

        elif to_train == GENERATOR and rel_epochs >= gen_epochs:
            rel_epochs = 0
            to_train = DISCRIMINATOR
            train_string = "DISC"
        
        # Change epoch ratio after intial 'leveling out'
        if epoch == epoch_threshold:
            rel_epochs = 0
            to_train = GENERATOR
            train_string = "GENERATOR"

            old_ratio = gen_epochs / disc_epochs
            gen_epochs = gen_epochs_change
            disc_epochs = disc_epochs_change
            new_ratio = gen_epochs / disc_epochs
            print(f'\n\nTraining ratio of G/D switched from {old_ratio:.{dig}f} to {new_ratio:.{dig}f}\n\n')
    else:
        if epoch < static_threshold:
            if to_train == DISCRIMINATOR and rel_epochs >= static_disc_epochs:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

            elif to_train == GENERATOR and rel_epochs >= static_gen_epochs:
                rel_epochs = 0
                to_train = DISCRIMINATOR
                train_string = "DISC"
        
        else:
            if to_train == DISCRIMINATOR and acc >= pull_threshold:
                to_train = GENERATOR
                train_string = "GEN"
            if to_train == GENERATOR and acc <= push_threshold:
                to_train = DISCRIMINATOR
                train_string = "DISC"

    print(f'Epoch [{epoch + 1}/{n_epochs}] Training: {train_string} ', end ='')
    cc, P, R, fpR, F1 = 0, 0, 0, 0, 0
    for batch_idx, (real_features, _) in enumerate(train_loader):
        #batch_size = len(real_features)
    
        if print_batches:
                print(f'\n\tBatch [{batch_idx + 1} / {len(train_loader)}] |', end ='')
    
        if to_train == DISCRIMINATOR:
            ### Training Discriminator
            #visualize_real_batch(real_features.float())
            opt_disc.zero_grad()
            disc_loss = get_disc_loss(gen, disc, criterion, real_features.float(), batch_size, z_dim)
            disc_loss.backward(retain_graph = True)
            opt_disc.step()
            acc, P, R, fpR, F1 = performance_stats(gen, disc, batch_size, batch = real_features.float())
            if print_batches:
                print(f'Loss D: {disc_loss.item():.digf}, Loss G: {get_gen_loss(gen, disc, criterion, batch_size, z_dim):.{dig}f} | Accuracy: {acc:.{dig}f} | fpR: {fpR:.{dig}f} P: {P:.{dig}f} | R: {R:.{dig}f} | F1: {F1:.{dig}f}')
        else:
            ### Training Generator
            
            opt_gen.zero_grad()
            gen_loss = get_gen_loss(gen, disc, criterion, batch_size, z_dim)
            gen_loss.backward()
            opt_gen.step()
            acc, P, R, fpR, F1 = performance_stats(gen, disc, batch_size, batch = real_features.float())
            if print_batches:
                print(f'Loss D: {get_disc_loss(gen, disc, criterion, real_features.float(), batch_size, z_dim):.{dig}f}, Loss G: {gen_loss.item():.{dig}f} | Accuracy: {acc:.{dig}f} | fpR: {fpR:.{dig}f} P: {P:.{dig}f} | R: {R:.{dig}f} | F1: {F1:.{dig}f}')
        
    if not print_batches:
        if to_train == DISCRIMINATOR:
            print(f'| Loss D: {disc_loss.item():.{dig}f}, Loss G: {get_gen_loss(gen, disc, criterion, batch_size, z_dim):.{dig}f} | Accuracy: {acc:.{dig}f} | fpR: {fpR:.{dig}f} P: {P:.{dig}f} | R: {R:.{dig}f} | F1: {F1:.{dig}f}')
        else:
            print(f'| Loss D: {get_disc_loss(gen, disc, criterion, real_features.float(), batch_size, z_dim):.{dig}f}, Loss G: {gen_loss.item():.{dig}f} | Accuracy: {acc:.{dig}f} | fpR: {fpR:.{dig}f} P: {P:.{dig}f} | R: {R:.{dig}f} | F1: {F1:.{dig}f}')
    rel_epochs += 1

Epoch [1/3000] Training: DISC | Loss D: 0.708, Loss G: 0.843 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [2/3000] Training: DISC | Loss D: 0.705, Loss G: 0.841 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [3/3000] Training: DISC | Loss D: 0.706, Loss G: 0.841 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [4/3000] Training: GEN | Loss D: 0.706, Loss G: 0.842 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [5/3000] Training: GEN | Loss D: 0.705, Loss G: 0.841 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [6/3000] Training: GEN | Loss D: 0.708, Loss G: 0.840 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [7/3000] Training: GEN | Loss D: 0.704, Loss G: 0.841 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [8/3000] Training: GEN | Loss D: 0.702, Loss G: 0.842 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [9/3000

Epoch [68/3000] Training: GEN | Loss D: 0.708, Loss G: 0.839 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [69/3000] Training: GEN | Loss D: 0.705, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [70/3000] Training: GEN | Loss D: 0.705, Loss G: 0.840 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [71/3000] Training: GEN | Loss D: 0.707, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [72/3000] Training: GEN | Loss D: 0.706, Loss G: 0.839 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [73/3000] Training: GEN | Loss D: 0.707, Loss G: 0.839 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [74/3000] Training: GEN | Loss D: 0.705, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [75/3000] Training: GEN | Loss D: 0.705, Loss G: 0.841 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [7

Epoch [134/3000] Training: GEN | Loss D: 0.708, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [135/3000] Training: GEN | Loss D: 0.708, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [136/3000] Training: GEN | Loss D: 0.709, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [137/3000] Training: GEN | Loss D: 0.707, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [138/3000] Training: GEN | Loss D: 0.705, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [139/3000] Training: GEN | Loss D: 0.708, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [140/3000] Training: GEN | Loss D: 0.704, Loss G: 0.839 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [141/3000] Training: GEN | Loss D: 0.705, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000


Epoch [200/3000] Training: GEN | Loss D: 0.705, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [201/3000] Training: GEN | Loss D: 0.704, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [202/3000] Training: GEN | Loss D: 0.707, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [203/3000] Training: GEN | Loss D: 0.706, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [204/3000] Training: GEN | Loss D: 0.708, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [205/3000] Training: GEN | Loss D: 0.709, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [206/3000] Training: GEN | Loss D: 0.707, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [207/3000] Training: GEN | Loss D: 0.707, Loss G: 0.839 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000


Epoch [266/3000] Training: GEN | Loss D: 0.707, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [267/3000] Training: GEN | Loss D: 0.703, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [268/3000] Training: GEN | Loss D: 0.707, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [269/3000] Training: GEN | Loss D: 0.710, Loss G: 0.838 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [270/3000] Training: GEN | Loss D: 0.707, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [271/3000] Training: GEN | Loss D: 0.705, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [272/3000] Training: GEN | Loss D: 0.706, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [273/3000] Training: GEN | Loss D: 0.706, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000


Epoch [332/3000] Training: GEN | Loss D: 0.707, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [333/3000] Training: GEN | Loss D: 0.709, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [334/3000] Training: GEN | Loss D: 0.709, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [335/3000] Training: GEN | Loss D: 0.711, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [336/3000] Training: GEN | Loss D: 0.708, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [337/3000] Training: GEN | Loss D: 0.712, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [338/3000] Training: GEN | Loss D: 0.709, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [339/3000] Training: GEN | Loss D: 0.710, Loss G: 0.837 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000


Epoch [398/3000] Training: GEN | Loss D: 0.710, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [399/3000] Training: GEN | Loss D: 0.708, Loss G: 0.833 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [400/3000] Training: GEN | Loss D: 0.708, Loss G: 0.833 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [401/3000] Training: GEN | Loss D: 0.707, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [402/3000] Training: GEN | Loss D: 0.708, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [403/3000] Training: GEN | Loss D: 0.708, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [404/3000] Training: GEN | Loss D: 0.707, Loss G: 0.835 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [405/3000] Training: GEN | Loss D: 0.709, Loss G: 0.833 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000


Epoch [465/3000] Training: GEN | Loss D: 0.708, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [466/3000] Training: GEN | Loss D: 0.705, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [467/3000] Training: GEN | Loss D: 0.710, Loss G: 0.833 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [468/3000] Training: GEN | Loss D: 0.709, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [469/3000] Training: GEN | Loss D: 0.708, Loss G: 0.836 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [470/3000] Training: GEN | Loss D: 0.710, Loss G: 0.834 | Accuracy: 0.654 | fpR: 0.000 P: 0.000 | R: 0.000 | F1: 0.000
Epoch [471/3000] Training: GEN 

KeyboardInterrupt: 

# Visualizing Generation Quality